In [18]:
import requests
from bs4 import BeautifulSoup
url = "https://www.linkedin.com/in/teddyoweh/recent-activity/all/"
headers = {
    "authority": "www.linkedin.com",
    "method": "GET",
    "scheme": "https",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9",
    "Cache-Control": "max-age=0",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-User": "?1",
    "upgrade-insecure-requests": "1",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36",
    "Cookie": open("cok.txt").read()
}

 
response = requests.get(url, headers=headers, verify=True)
soup = BeautifulSoup(response.content, 'html.parser')
code_tags = soup.find_all('code')
 
    

In [19]:
response.status_code


200

In [ ]:
data =soup.find('div',class_='scaffold-finite-scroll__content').find('ul').find_all('li',class_='profile-creator-shared-feed-update__container')

In [ ]:
post =data[0].find('div',id='fie-impression-container')
post.find('span', class_='update-components-actor__name').text.strip()


In [ ]:
data[3]

In [6]:
poster_name = post.find('span', class_='update-components-actor__name').text.strip()
poster_name

NameError: name 'post' is not defined

In [ ]:
def extract_post_data(soup):
    data =soup.find('div',class_='scaffold-finite-scroll__content').find('ul').find_all('li',class_='profile-creator-shared-feed-update__container')    
    print(len(data))
    posts = []
    for x in range(len(data)):
        try:
            item =data[x].find('div',id='fie-impression-container')
            post = {}
        
            post['poster_name'] = item.find('span', class_='update-components-actor__name').text.strip()
            post['post_text'] = item.find('div', class_='update-components-text relative update-components-update-v2__commentary').text.strip()
            post['date'] = item.find('span', class_='update-components-actor__sub-description').text.strip()
            post['images'] = []
            image_links = item.find_all('img', class_='ivm-view-attr__img--centered')
            for image in image_links:
                post['images'].append(image['src'])
            
            posts.append(post)
        except:
            pass
    return posts        
pxx =extract_post_data(soup)


In [ ]:
pxx

In [ ]:
data[1]

In [ ]:
api_key = "sk-proj-9nfIAOPBmcB102QxQlgpT3BlbkFJSmzfpZdBqdK2Ni4Ga42q"
from openai import OpenAI

def generate_pitch(posts, business_description):
    client = OpenAI(api_key=api_key)

    # Concatenate all post texts
    post_texts = [post['post_text'] for post in posts]
    all_posts_text = ' '.join(post_texts)

 
    images = [image for post in posts for image in post.get('images', [])]
    
 
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": f"write a fucking authentic,   pitch to the user on this,{business_description}: keep it fucking concise, sound realistic as fuck and have just fucking insane charisma"
            },
            {
                "role": "user",
                "content": [
                                   {"type": "text", "text": "use his previous posts on linkedlin here as content"},
                      {"type": "text", "text": str(post_texts)},
                                                         {"type": "text", "text": "do not sound like a fucking ai"},

                ]+[
                     {
          "type": "image_url",
          "image_url": {
            "url": urlx,
          },
        } for urlx in images
                ]
            },
           
        ],
        max_tokens=300,
    )

    return response.choices[0].message.content

 
 
business_description = "saas business that improves product managers productivity"
pitch = generate_pitch(pxx, business_description)
open('pitch.txt','w').write(pitch)